In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bertviz import head_view

# Path to your fine-tuned model
model_path = '/datasets/mdawood/swahili-xlmr-binary-classification-1mil'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, output_attentions=True)

# Sample Swahili text (replace with your own example)
text = "Leo, rais alitoa hotuba muhimu kuhusu maendeleo ya nchi."

# Tokenize the input
inputs = tokenizer(text, return_tensors='pt')

# Get model outputs
outputs = model(**inputs)

# Extract attention
attention = outputs.attentions

# Get tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Generate and save the head view visualization
html_head_view = head_view(attention, tokens, html_action='return')

with open('head_view.html', 'w', encoding='utf-8') as f:
    f.write(html_head_view.data)

print("Head View saved to head_view.html")

# Display the visualization in the notebook
from IPython.display import display
display(html_head_view)

Head View saved to head_view.html
